# Run a test

Start exploring how metrics and threshold tests work by running one of the tests available out-of-the-box and logging the results to your model documentation.

What are metrics or tests?

- **Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

- **Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

This interactive notebook provides a step-by-step guide for loading a demo dataset, preprocessing the raw data, training a model for testing, settting up test inputs, initializing the required ValidMind objects, running the test, and then logging the results to ValidMind.

## Contents<a id='toc0_'></a>    
- [About ValidMind](#toc1_)    
  - [Before you begin](#toc1_1_)    
  - [New to ValidMind?](#toc1_2_)    
  - [Key concepts](#toc1_3_)    
- [Install the client library](#toc2_)    
- [Initialize the client library](#toc3_)    
- [Load the demo dataset](#toc4_)    
  - [Prepocess the raw dataset](#toc4_1_)    
- [Train a model for testing](#toc5_)    
- [Set up test inputs and run the test](#toc6_)    
  - [Initialize ValidMind objects](#toc6_1_)    
  - [Assign predictions to the datasets](#toc6_2_)    
  - [Run the test](#toc6_3_)    
  - [Log the test results to ValidMind](#toc6_4_)    
- [Next steps](#toc7_)    
  - [Work with your model documentation](#toc7_1_)    
  - [Discover more learning resources](#toc7_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>[About ValidMind](#toc0_)

ValidMind is a platform for managing model risk, including risk associated with AI and statistical models. You use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

### <a id='toc1_1_'></a>[Before you begin](#toc0_)

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

### <a id='toc1_2_'></a>[New to ValidMind?](#toc0_)

If you haven't already seen our [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html), we recommend you explore the available resources for developers at some point. There, you can learn more about documenting models, find code samples, or read our developer reference.

::: {.callout-tip}

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)

:::

### <a id='toc1_3_'></a>[Key concepts](#toc0_)

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with ValidMind to be used in the platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind framework. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](../validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](../validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](../validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.


## <a id='toc2_'></a>[Install the client library](#toc0_)


In [ ]:
%pip install -q validmind

## <a id='toc3_'></a>[Initialize the client library](#toc0_)

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select **Binary classification** as the template and **Marketing/Sales - Attrition/Churn Management** as the use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="...",
)

In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## <a id='toc4_'></a>[Load the demo dataset](#toc0_)


In [ ]:
from validmind.datasets.classification import customer_churn as demo_dataset

# You can also import customer_churn like this:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

### <a id='toc4_1_'></a>[Prepocess the raw dataset](#toc0_)


In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

## <a id='toc5_'></a>[Train a model for testing](#toc0_)

We train a simple customer churn model for our test.


In [ ]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

## <a id='toc6_'></a>[Set up test inputs and run the test](#toc0_)

### <a id='toc6_1_'></a>[Initialize ValidMind objects](#toc0_)


In [ ]:
vm_train_ds = vm.init_dataset(
    input_id="train_dataset",
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column,
)

vm_test_ds = vm.init_dataset(
    input_id="test_dataset",
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column,
)

vm_model = vm.init_model(model, input_id="model")

### <a id='toc6_2_'></a>[Assign predictions to the datasets](#toc0_)

We can now use the `assign_predictions()` method from the `Dataset` object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:


In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
)
vm_test_ds.assign_predictions(
    model=vm_model,
)

### <a id='toc6_3_'></a>[Run the test](#toc0_)

Individual tests can be easily run by calling the `run_test` function provided by the `validmind.tests` module. The function takes the following arguments:

- `test_id`: The ID of the test to run. To find a particular test and get its, refer to the `explore_tests.ipynb` notebook
- `params`: A dictionary of parameters for the test. These will override any `default_params` set in the test definition. Refer to the `explore_tests.ipynb` notebook to find the default parameters for a test.

You can then pass in any inputs for the test as keyword arguments. Most likely, these will be `dataset` and `model` objects. Again, you may refer to the `explore_tests.ipynb` notebook to find the required inputs for a test.


In [ ]:
test = vm.tests.run_test(
    test_id="validmind.model_validation.sklearn.TrainingTestDegradation",
    params={},  # can be used to set overrides to the test's default parameters
    inputs={"model": vm_model, "datasets": (vm_train_ds, vm_test_ds)},
)

### <a id='toc6_4_'></a>[Log the test results to ValidMind](#toc0_)

After the test has been run, you can save the results to ValidMind by calling the `log` method of the test object returned after running the test:


In [ ]:
test.log()

## <a id='toc7_'></a>[Next steps](#toc0_)

You can look at the output produced by the ValidMind Developer Framework right in the notebook where you ran the code, as you would expect. But there is a better way: use the ValidMind platform to work with your model documentation.

### <a id='toc7_1_'></a>[Work with your model documentation](#toc0_)

1. From the [**Model Inventory**](https://app.prod.validmind.ai/model-inventory) in the ValidMind Platform UI, go to the model you registered earlier.

2. Click and expand the **Model Development** section.

What you see is the full draft of your model documentation in a more easily consumable version. From here, you can make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready. [Learn more ...](https://docs.validmind.ai/guide/working-with-model-documentation.html)

### <a id='toc7_2_'></a>[Discover more learning resources](#toc0_)

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](testing-overview.qmd)
- [Code samples](samples-jupyter-notebooks.qmd)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.